In [1]:
import numpy as np
from numpy import linalg as LA
import pandas as pd

import json
import keras
import tensorflow
import tensorflow_addons as tfa
from keras import layers, Model
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

Using TensorFlow backend.


In [2]:
csv_path = './UrbanSound8K/metadata/UrbanSound8K.csv'
embeddings_path = "./embeddings/"
model_save_path = "data_generation"
test_size = 0.2
val_size = 0.2
batch_size = 32
num_classes = 5

In [3]:
class EmbeddingsLoader(keras.utils.Sequence):
  def __init__(self, x_set, y_set, batch_size, emb_dir):
    self.x, self.y = x_set, y_set
    self.batch_size = batch_size
    self.emb_dir = emb_dir

  def __len__(self):
    return int(np.ceil(len(self.x) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) *
    self.batch_size]
    batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

    batchEmbs = []
    for fileName in batch_x:
        if fileName == "synthetic":
            emb = np.random.randint(256, size=128)
        else:
            emb = np.load(self.emb_dir + fileName + ".npy")[0]
        batchEmbs.append(emb)
    batchEmbs = np.array(batchEmbs)
    return batchEmbs, np.array(batch_y)

In [4]:
data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < num_classes]
data_df_unknown = data_df.loc[data_df["classID"] >= num_classes]

X_trainval, X_test, y_trainval, y_test = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=test_size, random_state = 42)

# Add synthetic data to known data
X_trainval.extend(["synthetic"]*len(X_trainval))
y_trainval.extend([num_classes]*len(y_trainval))

X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=val_size, random_state = 42)

# Add unknown data to test set
X_trash, X_unknown, y_trash, y_unknown = train_test_split(data_df_unknown['slice_file_name'].tolist(), data_df_unknown['classID'].tolist(), test_size=test_size, random_state = 42)
X_test = X_test + X_unknown
y_test.extend([num_classes]*len(y_unknown))

train_loader = EmbeddingsLoader(X_train, y_train, batch_size, embeddings_path)
test_loader = EmbeddingsLoader(X_test, y_test, batch_size, embeddings_path)
val_loader = EmbeddingsLoader(X_val, y_val, batch_size, embeddings_path)

In [5]:
model = Sequential()
model.add(layers.InputLayer(input_shape=(128,)))
model.add(layers.Dense(num_classes+1, activation="softmax"))
model.summary()
model.compile(
      optimizer="Adam",
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 774       
Total params: 774
Trainable params: 774
Non-trainable params: 0
_________________________________________________________________


In [6]:
earlystopping = EarlyStopping(
                    patience=10, 
                    restore_best_weights=True)
checkpoint = ModelCheckpoint(
                    model_save_path, 
                    monitor="val_accuracy", 
                    save_best_only=True)

model.fit(x=train_loader,
          validation_data=val_loader,
          callbacks=[checkpoint, earlystopping],
          epochs=200,
          verbose=1
         )

In [7]:
def evaluate(predicted, expected):
    acc = np.mean(np.array(predicted) == np.array(expected))
    print("Overall accuracy: {}".format(acc))
    acc_dict = {}
    for i in range(len(expected)):
        expected_class = expected[i]
        if expected_class not in acc_dict:
            acc_dict[expected_class] = [0, 0]
        acc_dict[expected_class][1] += 1
        if expected_class == predicted[i]:
            acc_dict[expected_class][0] += 1
    for k,v in acc_dict.items():
        print("Accuracy for class {}: {}".format(k, v[0]/v[1]))
    for average in ["macro", "weighted", "micro"]:
        f1 = f1_score(expected, predicted, average=average)
        print("{} f1 score: {}".format(average, f1))

In [8]:
model.load_weights(model_save_path)

In [9]:
predicted_val = model.predict_classes(x=val_loader, batch_size=None)
predicted_test = model.predict_classes(x=test_loader, batch_size=None)

In [10]:
evaluate(predicted_val, y_val)
evaluate(predicted_test, y_test)

Overall accuracy: 0.8808180535966149
Accuracy for class 0: 0.8053691275167785
Accuracy for class 5: 0.9763888888888889
Accuracy for class 2: 0.8709677419354839
Accuracy for class 3: 0.64375
Accuracy for class 4: 0.8164556962025317
Accuracy for class 1: 0.7763157894736842
macro f1 score: 0.8244368958493468
weighted f1 score: 0.8796253816023716
micro f1 score: 0.8808180535966149
Overall accuracy: 0.45163136805953064
Accuracy for class 2: 0.8423645320197044
Accuracy for class 4: 0.8546511627906976
Accuracy for class 0: 0.7751196172248804
Accuracy for class 3: 0.6447368421052632
Accuracy for class 1: 0.7837837837837838
Accuracy for class 5: 0.1207897793263647
macro f1 score: 0.5056223889884606
weighted f1 score: 0.3898216489817824
micro f1 score: 0.45163136805953064
